# Basic Summary Statistics

In [23]:
from pandas import read_csv

df = read_csv("https://raw.githubusercontent.com/prof-rossetti/python-for-finance/main/docs/data/monthly-indicators.csv")
df.head()

,timestamp,cpi,fed,spy,gld
0,2024-05-01,314.069,5.33,525.6718,215.30
1,2024-04-01,313.548,5.33,500.3636,211.87
2,2024-03-01,312.332,5.33,521.3857,205.72
3,2024-02-01,310.326,5.33,504.8645,189.31
4,2024-01-01,308.417,5.33,479.8240,188.45


In [24]:
print(len(df))
print(df["timestamp"].min(), "...", df["timestamp"].max())

234
2004-12-01 ... 2024-05-01


https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html

We can use the describe method to quickly see the basic summary statistics for each column:

In [25]:
df.describe()

,cpi,fed,spy,gld
count,234.000000,234.000000,234.000000,234.000000
mean,239.904709,1.588761,199.248881,124.362344
std,30.299220,1.879657,122.947398,40.170234
min,190.300000,0.050000,55.148800,41.650000
25%,216.963500,0.120000,98.790475,101.575000
50%,236.409000,0.390000,164.878300,123.360000
75%,256.327500,2.480000,270.226075,159.787500
max,314.069000,5.330000,525.671800,215.300000


As you may be aware, we can calculate these individually, using `Series` aggregations:

In [52]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.html
# https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html

series = df["fed"]

print("COUNT:", len(series))
print("MEAN:", series.mean().round(6))
print("STD:", series.std().round(6))
print("-------------")
print("MIN:", series.min())
print("25TH:", series.quantile(.25))
print("MED:", series.median())
print("75TH:", series.quantile(.75))
print("MAX:", series.max())

COUNT: 234
MEAN: 1.588761
STD: 1.879657
-------------
MIN: 0.05
25TH: 0.12
MED: 0.39
75TH: 2.48
MAX: 5.33


In [27]:
series.describe() # for comparison

count    234.000000
mean       1.588761
std        1.879657
min        0.050000
25%        0.120000
50%        0.390000
75%        2.480000
max        5.330000
Name: fed, dtype: float64

## Distribution Plots

Let's view some distribution plots of the federal funds rate, to tell a story about the summary statistics for this indicator.

In [28]:
import plotly.express as px

px.box(df, x="fed", orientation="h", points="all", title="Distribution of Federal Funds Rate (Monthly)", hover_data=["timestamp"],)

In [29]:
# https://plotly.com/python-api-reference/generated/plotly.express.violin.html
import plotly.express as px

#px.violin(df, y="fed", points="all", box=True, title="Distribution of Federal Funds Rate (Monthly)", hover_data=["timestamp"])
px.violin(df, x="fed", orientation="h", points="all", box=True, title="Distribution of Federal Funds Rate (Monthly)", hover_data=["timestamp"])

In [30]:
# https://plotly.com/python-api-reference/generated/plotly.express.histogram.html
px.histogram(df, x="fed", #nbins=12,
        title="Distribution of Federal Funds Rate (Monthly)", height=350)

Looks like the recent higher funds rates are potential outliers. It is hard to say for sure if this data is normally distributed, or whether it is too skewed by the outliers.

# Statistical Tests with `Scipy`


We can use the Scipy package to perform basic statistical tests.

https://pypi.org/project/scipy/


## Normality Tests

We can conduct a normality test to see if a given distribution is normally distributed.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html

> This function tests the null hypothesis that a sample comes from a normal distribution.
>
> If the p-value is "small" - that is, if there is a low probability of sampling data from a normally distributed population that produces such an extreme value of the statistic - this may be taken as evidence against the null hypothesis in favor of the alternative: the weights were not drawn from a normal distribution.

In [31]:
from scipy.stats import normaltest

x = df["fed"]

result = normaltest(x)
print(result)

NormaltestResult(statistic=34.68795952886342, pvalue=2.9349809995776456e-08)


Interpreting the results.

https://support.minitab.com/en-us/minitab/21/help-and-how-to/statistics/basic-statistics/how-to/normality-test/interpret-the-results/key-results/

> To determine whether the data do not follow a normal distribution, compare the p-value to the significance level. Usually, a significance level (denoted as α or alpha) of 0.05 works well. A significance level of 0.05 indicates a 5% risk of concluding that the data do not follow a normal distribution when the data do follow a normal distribution.
>
> P-value ≤ α: The data do not follow a normal distribution (Reject H0)
> If the p-value is less than or equal to the significance level, the decision is to reject the null hypothesis and conclude that your data do not follow a normal distribution.
>
> P-value > α: You cannot conclude that the data do not follow a normal distribution (Fail to reject H0). If the p-value is larger than the significance level, the decision is to fail to reject the null hypothesis. You do not have enough evidence to conclude that your data do not follow a normal distribution.

In [32]:
if result.pvalue <= 0.05:
    print("REJECT (NOT NORMAL)")
else:
    print("NOT ABLE TO REJECT (COULD BE NORMAL)")

REJECT (NOT NORMAL)


Looks like the federal fuds rate does not have a normal distribution (as this notebook was run on June 28th 2024).

How about the market?

In [33]:

x = df["spy"]

result = normaltest(x)
print(result)

if result.pvalue <= 0.05:
    print("REJECT (NOT NORMAL)")
else:
    print("NOT ABLE TO REJECT (COULD BE NORMAL)")

NormaltestResult(statistic=27.560328618235523, pvalue=1.0359783530157106e-06)
REJECT (NOT NORMAL)


## T-Tests


https://www.investopedia.com/terms/t/t-test.asp

> A t-test is an inferential statistic used to determine if there is a significant difference between the means of two groups and how they are related. T-tests are used when the data sets follow a normal distribution and have unknown variances, like the data set recorded from flipping a coin 100 times.

### T-Test Considerations

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6676026/#sec-2title

In order to conduct a T-Test, the data needs to be normally distributed. So the examples below may not be the most methodologically sound. However they should provide code examples you can adapt for other use cases in the future.

### 2 Sample T-Test

A two sample T-test is used to determine whether two independent samples come from the same distribution.


Let's split the most recent year's rates from the rest. And see if the most recent years are statistically different.

In [53]:

#cutoff_date = "2022-06-01" # you can chose a different one if you'd like
cutoff_date = "2022-10-01"

rates_recent = df[df["timestamp"] >= cutoff_date]["fed"]
print(len(rates_recent))
print(rates_recent)

20
0     5.33
1     5.33
2     5.33
3     5.33
4     5.33
5     5.33
6     5.33
7     5.33
8     5.33
9     5.33
10    5.12
11    5.08
12    5.06
13    4.83
14    4.65
15    4.57
16    4.33
17    4.10
18    3.78
19    3.08
Name: fed, dtype: float64


In [54]:
rates_historic = df[df["timestamp"] < cutoff_date]["fed"]
print(len(rates_historic))
print(rates_historic)

214
20     2.56
21     2.33
22     1.68
23     1.21
24     0.77
       ... 
229    2.79
230    2.63
231    2.50
232    2.28
233    2.16
Name: fed, Length: 214, dtype: float64


https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

> Calculate the T-test for the means of two independent samples of scores.
>
> This is a test for the null hypothesis that 2 independent samples have identical average (expected) values. This test assumes that the populations have identical variances by default.
>
> The t-test quantifies the difference between the arithmetic means of the two samples. The p-value quantifies the probability of observing as or more extreme values assuming the null hypothesis, that the samples are drawn from populations with the same population means, is true. A p-value larger than a chosen threshold (e.g. 5% or 1%) indicates that our observation is not so unlikely to have occurred by chance. Therefore, we do not reject the null hypothesis of equal population means. If the p-value is smaller than our threshold, then we have evidence against the null hypothesis of equal population means.

In [35]:
print(rates_recent.var())
print(rates_historic.var())

0.4033105263157895
2.7065506493791407


In [36]:
from scipy.stats import ttest_ind

result = ttest_ind(rates_recent, rates_historic)
print(result)

if result.pvalue <= 0.05:
    print("REJECT (MEANS NOT THE SAME)")
else:
    print("NOT ABLE TO REJECT (MEANS COULD BE THE SAME)")

TtestResult(statistic=9.743816217891522, pvalue=5.021356895595338e-19, df=232.0)
REJECT (MEANS NOT THE SAME)


### 1 Sample T-Test

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html


> Calculate the T-test for the mean of ONE group of scores.
>
> This is a test for the null hypothesis that the expected value (mean) of a sample of independent observations a is equal to the given population mean, popmean.
>
> Under certain assumptions about the population from which a sample is drawn, the confidence interval with confidence level 95% is expected to contain the true population mean in 95% of sample replications.

Suppose we wish to test the null hypothesis that the mean of the fed funds rates is equal to 2.5%.


In [37]:
from scipy.stats import ttest_1samp

x = df["fed"]
print(x.mean())

popmean = 2.5 # for example
result = ttest_1samp(x, popmean=popmean)
print(result)

if result.pvalue <= 0.05:
    print("REJECT (MEAN NOT EQUAL TO POPMEAN)")
else:
    print("NOT ABLE TO REJECT (MEAN COULT BE EQUAL TO POPMEAN)")

ci = result.confidence_interval(confidence_level=0.95)
print(ci)

1.5887606837606836
TtestResult(statistic=-7.415864219982758, pvalue=2.2306437030862214e-12, df=233)
REJECT (MEAN NOT EQUAL TO POPMEAN)
ConfidenceInterval(low=1.346668668631088, high=1.8308526988902791)
